# Problema
As distribuições dos dados mudam. Evento conhecido como "mudança de conceito". Aplicações práticas precisam lidar com isso. Como detectar a mudança? Como se adaptar à mudança? 

## Dados estacionários

In [ ]:
# gráfico da performance convergindo (theta1 = theta2)

## Dados não estacionários (com concept drift)

In [ ]:
# gráfico da performance baixando (theta1 != theta2)

# Objetivo
Identificar bases de dados com mudanças de conceito para servirem de amostra na avaliação de classificadores com aprendizagem online adaptativa.

# Metodologia

Executar experimento com um classificador não adaptativo (Naive Bayes) e monitorar a performance dele. Uma piora na performance desse classificador corresponde a um evento de mudança de conceito.
Como monitorar? Teste de hipótese em amostras definidas por intervalos de tempo? Ou um procedimento com monitoramento contínuo?
Page Hinkley Test viabiliza o monitoramente contínuo com baixo custo computacional e baixo uso de memória por não precisar reprocessar registros passados e manter janelas de tempo na memória.

## Hipóteses
H0: theta1 = theta2
<br/>
H1: theta1 != theta2

## Métrica
Recall = TP/(TP+FN)
Um recall para cada classe.

# Resultados

# Conclusão